In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import re
import nltk
import torch
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
r= requests.get('https://www.yelp.com/biz/taco-bell-san-francisco-12')
soup= BeautifulSoup(r.text, 'html.parser')
regex= re.compile('.*comment*.')
results= soup.find_all('p', {'class':regex})
reviews= [result.text for result in results]

In [3]:
r.text

'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>\n            window.yelp = window.yelp || {};\

In [4]:
results

[<p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">mario and the manager hunter were so amazing!! Mario made great drinks and so attentive. Hunter the manager was great as well!!</span></p>,
 <p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">Yum ba hahah blastie <br/> <br/>poo</span></p>,
 <p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">Taco Bells and McDonalds..hahah KFC fried chicken is pretty good and Taco Bell is not at par...however sometimes it's fun to get Taco Bell like Saved by the Bell..ahaha has some good items to play with your food. Old jokes.</span></p>,
 <p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">I love the new cheeze it Crunchwrap! So yummy! I even got a frozen Baja blast margarita to wash down the cheesy goodness !</span></p>,
 <p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="

In [5]:
reviews

['mario and the manager hunter were so amazing!! Mario made great drinks and so attentive. Hunter the manager was great as well!!',
 'Yum ba hahah blastie  poo',
 "Taco Bells and McDonalds..hahah KFC fried chicken is pretty good and Taco Bell is not at par...however sometimes it's fun to get Taco Bell like Saved by the Bell..ahaha has some good items to play with your food. Old jokes.",
 'I love the new cheeze it Crunchwrap! So yummy! I even got a frozen Baja blast margarita to wash down the cheesy goodness !',
 "Ripped my bootyhole. Don't recommend at all. I have 3rd degree burns. Also I had to throw away my table when I was done. 10/10 dining experience. Sounds pretty loud. Food ripped my bootyhole open.",
 "I really don't understand what's so difficult about getting an order right. They NEVER do. They also never include hot sauce on orders even when specially added to the order in the app. It's not rocket science. This place is no good.",
 "Good old taco bell always satisfies! Can't

In [6]:
df= pd.DataFrame(reviews, columns= ['reviews'])

In [7]:
df.head()

reviews
0  mario and the manager hunter were so amazing!!...
1                          Yum ba hahah blastie  poo
2  Taco Bells and McDonalds..hahah KFC fried chic...
3  I love the new cheeze it Crunchwrap! So yummy!...
4  Ripped my bootyhole. Don't recommend at all. I...

In [8]:
df['lower']= df.reviews.str.lower()
df['lower']= [str(data) for data in df.lower]
df['lower']= df.lower.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x))

In [9]:
token_review= [word_tokenize(str(word)) for word in df.lower]

In [10]:
token_review

[['mario',
  'and',
  'the',
  'manager',
  'hunter',
  'were',
  'so',
  'amazing',
  'mario',
  'made',
  'great',
  'drinks',
  'and',
  'so',
  'attentive',
  'hunter',
  'the',
  'manager',
  'was',
  'great',
  'as',
  'well'],
 ['yum', 'ba', 'hahah', 'blastie', 'poo'],
 ['taco',
  'bells',
  'and',
  'mcdonalds',
  'hahah',
  'kfc',
  'fried',
  'chicken',
  'is',
  'pretty',
  'good',
  'and',
  'taco',
  'bell',
  'is',
  'not',
  'at',
  'par',
  'however',
  'sometimes',
  'it',
  's',
  'fun',
  'to',
  'get',
  'taco',
  'bell',
  'like',
  'saved',
  'by',
  'the',
  'bell',
  'ahaha',
  'has',
  'some',
  'good',
  'items',
  'to',
  'play',
  'with',
  'your',
  'food',
  'old',
  'jokes'],
 ['i',
  'love',
  'the',
  'new',
  'cheeze',
  'it',
  'crunchwrap',
  'so',
  'yummy',
  'i',
  'even',
  'got',
  'a',
  'frozen',
  'baja',
  'blast',
  'margarita',
  'to',
  'wash',
  'down',
  'the',
  'cheesy',
  'goodness'],
 ['ripped',
  'my',
  'bootyhole',
  'don',
  't'

In [11]:
stopwords= nltk.corpus.stopwords
stop_words= stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
count= CountVectorizer(tokenizer= word_tokenize, stop_words= stop_words, ngram_range=(1, 1))

In [13]:
count

CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<function word_tokenize at 0x7948b586cdc0>)

In [17]:
tokenizer= AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model= AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [21]:
def sentiment_score_of_reviews(review):
    tokens= tokenizer.encode(review, return_tensors= 'pt')
    result= model(tokens)
    return int(torch.argmax(result.logits))+1

In [22]:
df['sentiment']= df['reviews'].apply(lambda x: sentiment_score_of_reviews(x[:512]))

In [25]:
df # Sentiment Score 1-5. 1 being (very bad) and 5 being (very good)

reviews  \
0  mario and the manager hunter were so amazing!!...   
1                          Yum ba hahah blastie  poo   
2  Taco Bells and McDonalds..hahah KFC fried chic...   
3  I love the new cheeze it Crunchwrap! So yummy!...   
4  Ripped my bootyhole. Don't recommend at all. I...   
5  I really don't understand what's so difficult ...   
6  Good old taco bell always satisfies! Can't go ...   
7  The employees at this store are extremely lazy...   
8  Amazing Food! One of the bestGreat service! I ...   
9  Horrible service took 20 minutes to get food. ...   

                                               lower  sentiment  
0  mario and the manager hunter were so amazing  ...          5  
1                          yum ba hahah blastie  poo          5  
2  taco bells and mcdonalds hahah kfc fried chick...          3  
3  i love the new cheeze it crunchwrap  so yummy ...          5  
4  ripped my bootyhole  don t recommend at all  i...          1  
5  i really don t understand what s so difficult ...          1  
6  good old taco bell always satisfies  can t go ...          5  
7  the employees at this store are extremely lazy...          1  
8  amazing food  one of the bestgreat service  i ...          5  
9  horrible service took 20 minutes to get food  ...          1